In [1]:
import os
import pandas as pd
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from tagger.core.mongo.models.order import Order, OrderType, Payment
from tagger.models.inventory import Inventory, InventoryStatus
from tagger.core.label.print_label import print_label
from tagger.core.label.shipping_label import make_shipping_label
from tagger.core.label.receiving_label import make_receiving_label
from bson import ObjectId

def ready_for_shipping(order_id: str, inventory: Inventory):
    order = Order.objects(id=ObjectId(order_id)).first()
    inventory.out_order_id = order_id
    inventory.status = InventoryStatus.SHIPPING_PENDING
    inventory.save()
    print_label(make_shipping_label(order, inventory))



ImportError: cannot import name 'make_shipping_label' from 'tagger.core.label.shipping_label' (/Users/stephenj/alloff-backoffice-server/tagger/core/label/shipping_label.py)

In [7]:
in_stock_query = Inventory.objects.filter(status=InventoryStatus.IN_STOCK).order_by("id")
inventories = in_stock_query.all()
orders = (
            Order.objects(
                orderstatus__in=[
                    "PAYMENT_FINISHED",
                    "PRODUCT_PREPARING",
                    "DELIVERY_PREPARING",
                ],
            )
            .order_by("id")
            .all()
        )
inventory_map = {}
exhausted_inventory_ids = []
for inv in inventories:
    key = (inv.product_name.replace(" ", "") + "-" + inv.product_brand, inv.size.replace(" ", ""))
    if key in inventory_map:
        inventory_map[key].append(inv)
    else:
        inventory_map[key] = [inv]
        
order_item_inventory_matches = []

for order in orders:
    # Skip if already fully fulfilled
    if len(order.orders) == Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).count():
        continue
    
    payment = order.payment
    
    
    for item in order.orders:
        p = item.alloffproduct if item.alloffproduct is not None else item.product

        # Directly matching inventory
        direct_matches = [x for x in in_stock_query.filter(
            product_id=p._id
        ).exclude(
            id__in=exhausted_inventory_ids
        ).all() if x.size.replace(" ", "") == item.size.replace(" ", "")]
        
        if len(direct_matches) > 0:
            match = direct_matches[0]
            exhausted_inventory_ids.append(match.id)
            order_item_inventory_matches.append((order, item, match))
        
        # No direct matches --- start string matching
        else:
            stringmatch_key = (p.name.replace(" ", "") + "-" + p.brand.keyname, item.size.replace(" ", ""))
#             print(stringmatch_key)
            if stringmatch_key in inventory_map:
                # Matches excluding exhausted inventories
                matches = [x for x in inventory_map[stringmatch_key] if x.id not in exhausted_inventory_ids]
                
                # If all are exhausted, pass
                if len(matches) == 0:
                    pass
                # Match found!
                else:
                    match = matches[0]
                    exhausted_inventory_ids.append(match.id)
                    order_item_inventory_matches.append((order, item, match))
            else:
                # no hope
                pass
            

df_items = []
for order, item, inv in order_item_inventory_matches:
    payment = order.payment
    df_items.append({
        "order_id": order.id,
        "orderstatus": order.orderstatus,
        "buyername": payment.buyername,
        "ordername": payment.name,
        "size": inv.size,
        "ordered": order.created.date(),
        "inventory_id": inv.id,
        "inventory_code": inv.code,
    })

In [8]:
df = pd.DataFrame(df_items)
# df = df.sort_values(by="buyername")
df

,order_id,orderstatus,buyername,ordername,size,ordered,inventory_id,inventory_code
0,618d1f7526685ff05175627e,DELIVERY_PREPARING,인지현,클래식 롱 다운 코트,95,2021-11-11,242,211117-019


In [7]:
for order, item, inv in order_item_inventory_matches:
    ready_for_shipping(order.id, inv)


In [8]:
inventory_map

{('[Re-order]알파카블렌드니트가디건(스모크핑크)-TIME',
  '90(F)'): [<Inventory: Inventory #25 [IN_STOCK] [Re-order] 알파카 블렌드 니트 가디건(스모크 핑크) (201104)>],
 ('[기획]핸드메이드벨티드코트-SJSJ',
  '82(55)'): [<Inventory: Inventory #26 [IN_STOCK] [기획] 핸드메이드 벨티드 코트 (201104)>],
 ('[빈폴스포츠]듀얼후드코어쇼트다운(옐로우)-BEANPOLEL',
  '85'): [<Inventory: Inventory #27 [IN_STOCK] [빈폴스포츠] 듀얼 후드 코어 쇼트 다운(옐로우) (201104)>],
 ('리브블록후드구스다운점퍼-MINE',
  '82(55)'): [<Inventory: Inventory #29 [IN_STOCK] 리브 블록 후드 구스다운 점퍼 (201104)>],
 ('볼륨소매니트풀오버(민트)-DEWL',
  'FREE(99)'): [<Inventory: Inventory #30 [IN_STOCK] 볼륨 소매 니트 풀오버(민트) (201104)>],
 ('스티치디테일핸드메이드코트(그린)-CCCOLLECT',
  '88(66)'): [<Inventory: Inventory #31 [IN_STOCK] 스티치 디테일 핸드메이드 코트(그린) (201104)>],
 ('스플릿드레스-TIME',
  '82(55)'): [<Inventory: Inventory #32 [IN_STOCK] 스플릿 드레스 (201104)>],
 ('아노락퀼팅점퍼(오렌지)-THECASHMERE',
  '82(F)'): [<Inventory: Inventory #33 [IN_STOCK] 아노락 퀼팅 점퍼(오렌지) (201104)>],
 ('워싱코튼패딩점퍼(그레이)-CCCOLLECT',
  '82(55)'): [<Inventory: Inventory #35 [IN_STOCK] 워싱 코튼 패딩 점퍼(그레이) (201104)>],
 ('캐

In [3]:
ready_for_shipping("6178d20c17fc2e5a1f9ea805", Inventory.objects.get(code="211111-006-NP"))

In [4]:
print_label(make_receiving_label(Inventory.objects.get(id=243)))

<?xml version="1.0" encoding="utf-8"?>
<DesktopLabel Version="1">
  <DYMOLabel Version="3">
    <Description>DYMO Label</Description>
    <Orientation>Landscape</Orientation>
    <LabelName>Addresss0722370</LabelName>
    <InitialLength>0</InitialLength>
    <BorderStyle>SolidLine</BorderStyle>
    <DYMORect>
      <DYMOPoint>
        <X>0.23</X>
        <Y>0.06</Y>
      </DYMOPoint>
      <Size>
        <Width>3.21</Width>
        <Height>0.9966666</Height>
      </Size>
    </DYMORect>
    <BorderColor>
      <SolidColorBrush>
        <Color A="1" R="0" G="0" B="0"></Color>
      </SolidColorBrush>
    </BorderColor>
    <BorderThickness>1</BorderThickness>
    <Show_Border>False</Show_Border>
    <DynamicLayoutManager>
      <RotationBehavior>ClearObjects</RotationBehavior>
      <LabelObjects>
        <TextObject>
          <Name>InventoryCode</Name>
          <Brushes>
            <BackgroundBrush>
              <SolidColorBrush>
                <Color A="0" R="0" G="0" B="0"></C

[<Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>, <Order: Order object>]